In [13]:
import os
from getpass import getpass

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = "https://eu.api.smith.langchain.com "
os.environ['LANGSMITH_API_KEY'] =  os.getenv('LANGSMITH_API_KEY') or getpass('Enter your LangSmith API Key: ')
os.environ['LANGSMITH_PROJECT'] = 'LangChain-LangSmith-Demo'

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter GOOGLE API Key: "
)

By default, LangChain will trace LLM calls, chains, etc. 

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",       # choose appropriate model name
    temperature=0.0
    # you may add other parameters like max_tokens, etc.
)



In [4]:
def invoke_llm(messages):
    return llm.invoke(messages)

In [5]:
invoke_llm("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--ed55ccae-8307-409b-815b-34099f30a106-0', usage_metadata={'input_tokens': 2, 'output_tokens': 32, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 23}})

However, it won't capture functions from outside of LangChain. LangSmith can trace functions that are not part of LangChain, we just need to add the `@traceable` decorator. Let's try this for a few simple functions.

In [9]:
from langsmith import traceable
import random
import time

@traceable
def generate_random_number():
    return random.randint(0, 100)

@traceable
def generate_string_delay(input_string: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_string} ({number})"

@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("An error occurred!")
    else:
        return "No Error!"

In [10]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello")
    try:
        random_error()
    except ValueError as e:
        pass

100%|██████████| 10/10 [00:34<00:00,  3.40s/it]


We can also modify our traceable names if we'd like to make them more readable inside the UI.

In [11]:
@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

In [15]:
for _ in tqdm(range(2)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

100%|██████████| 2/2 [00:01<00:00,  1.99it/s]
